In [91]:
import sounddevice
import numpy as np
import time
import yaml
from scipy import constants

In [86]:
phonon_mesh_filepath = './data/BaS_Fm3m/mesh.yaml'
sample_rate = 44100
timelength = 5  # length of sample in seconds
min_audible = 20# minimum audible frequency in herz
max_audible = 8E3 # maximum audible frequency in herz
min_phonon = 0 # minimum phonon frequency in THz, if not set it is extracted from the phonon calc
max_phonon = 16 # maximum phonon frequency in THz, if not set it is extracted from the phonon calc

In [112]:
# this function adapted from
# https://stackoverflow.com/questions/73494717/trying-to-play-multiple-frequencies-in-python-sounddevice-blow-horrific-speaker
def callback(outdata: np.ndarray, frames: int, time, status) -> None:
    """writes sound output to 'outdata' from sound_queue."""
    result = None

    for frequency, start_index in audible_dictionary.items():
        t = (start_index + np.arange(frames)) / sample_rate
        t = t.reshape(-1, 1)

        wave = np.sin(2 * np.pi * frequency * t)

        if result is None:
            result = wave
        else:
            result += wave

        audible_dictionary[frequency] += frames

    if result is None:
        result = np.arange(frames) / sample_rate
        result = result.reshape(-1, 1)

    outdata[:] = result
    
def phonon_to_audible(phonon_frequencies):
    """takes phonon frequencies (in THz) and returns suitable phonon frequencies (in Hz)"""
    
    if len(phonon_frequencies) == 1:
        audible_frequencies = [440]
        print("only one phonon frequency, so mapping to 440Hz")
    else:
        audible_frequencies = linear_map(phonon_frequencies)
        
    return audible_frequencies

def linear_map(phonon_frequencies):
    """linearly maps phonon frequencies (in THz) to frequencies in the audible range (in Hz)"""
    
    if min_phonon is None:
        min_phonon_hz = min(phonon_frequencies)*1E12
    else:
        min_phonon_hz = min_phonon*1E12

    if max_phonon is None:
        max_phonon_hz = max(phonon_frequencies)*1E12
    else:
        max_phonon_hz = max_phonon*1E12        

    phonon_frequencies_hz = np.array(phonon_frequencies)*1E12
    scale_factor = (max_audible - min_audible) / (max_phonon_hz - min_phonon_hz)
    audible_frequencies = [ scale_factor*(frequency-min_phonon_hz) + min_audible for frequency in phonon_frequencies_hz]
    print("audible frequencies are (Hz):", audible_frequencies)
    
    return audible_frequencies

def frequencies_from_mesh(phonon_mesh_filepath):
    """return phonon frequencies at gamma point from a phonopy mesh.yaml. Assumes gamma point is zeroth
    element in data['phonon']."""
    
    # read yaml data 
    with open(phonon_mesh_filepath) as f:
        data = yaml.safe_load(f)

    # extract list of unique frequencies - these are in THz
    phonon_frequencies = list(set([dictionary['frequency'] for dictionary in data['phonon'][0]['band']]))
    phonon_frequencies = process_imaginary(phonon_frequencies)
    
    return phonon_frequencies
    
def process_imaginary(phonon_frequencies):
    # remove any imaginary modes
    phonon_cleaned_frequencies = [frequency for frequency in phonon_frequencies if frequency > 0]
    
    return phonon_cleaned_frequencies

def frequencies_from_mp_id(mp_id):
    """return phonon frequencies at gamma point from for a material hosted on the Materials Project ().
    Material is identified using unique ID number. Note that to use this feature you need a Materials
    Project API key (https://materialsproject.org/api)."""
    import mp_api
    from mp_api.client import MPRester

    with MPRester("os1XoXmCTeMm5rDO4kY9ClmfVKzuo5ek") as mpr:
        try:
            bs = mpr.phonon.get_data_by_id(mp_id).ph_bs
        except:
            print("this materials project entry does not appear to have phonon data")
            pass

    print("extracting frequencies for qpoint {}".format(bs.qpoints[0].cart_coords))

    phonon_frequencies = list(bs.bands[:,0])
    phonon_frequencies = process_imaginary(phonon_frequencies)
    print("phonon frequencies are (THz):", phonon_frequencies)
    
    return phonon_frequencies

def bose_einstien_distribution(energy,temperature):
    return 1 / (math.exp(energy/(constants.Boltzmann*temperature)) - 1)

def frequency_to_energy(frequency):
    """convert frequency in THz to energy in joules"""
    frequency_hz = frequency*1E12
    energy = constants.h*frequency_hz
    return energy
    
def excite_by_heat(phonon_frequencies, temperature):
    """return frequencies which have average occupation >= 1 at a given temperature. 
    Average occupation is calculated using Bose-Einstien statistics"""
    average_occupations = np.array([bose_einstien_distribution(frequency_to_energy(frequency),temperature) 
                           for frequency in phonon_frequencies])
    # need phonon_frequency as numpy array
    phonon_frequencies = np.array(phonon_frequencies)
    # can now use trendy indexing over a boolean array
    occupied_modes = phonon_frequencies[average_occupations >= 1]
    return occupied_modes

def play_chord(timelength):
    """DRONE POWER"""
    
    # create stream
    stream = sounddevice.OutputStream(channels=2, blocksize=sample_rate, 
                samplerate=sample_rate, callback=callback)

    # start stream and keep running for timelength
    stream.start()
    time.sleep(timelength)
    stream.stop()
    stream.close()


In [114]:
# get phonon frequencies as numpy array
# phonon_frequencies = frequencies_from_mesh(phonon_mesh_filepath)
Cs3Sb = "mp-10378" 
K2TeCl6  = "mp-569149" 
BaAg2GeS4 = "mp-7394" 
CuCl = "mp-571386" 
CaPbF6 = "mp-19799" 

for mp_id in [Cs3Sb,K2TeCl6,BaAg2GeS4,CuCl,CaPbF6]:

    # get phonons (in THz)
    phonon_frequencies = frequencies_from_mp_id(mp_id)
    
    phonon_frequencies = excite_by_heat(phonon_frequencies,300)

    # convert phonon frequencies to something in the audible range (return in Hz)
    audible_frequencies = phonon_to_audible(phonon_frequencies)

    # create global dictionary containing frequencies as keys. This will be used in the output stream.
    audible_dictionary = dict.fromkeys(audible_frequencies, 0)

    # create output stream and run for set time
    play_chord(timelength)


Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

extracting frequencies for qpoint [0. 0. 0.]
phonon frequencies are (THz): [1.2009381258136025, 1.2009381260784766, 1.6454931375023374, 1.7595325890055091, 1.7595325890513807, 1.7595325891872349, 2.1939100118368193, 2.1939100119833475, 2.6240476164394857]
audible frequencies are (Hz): [618.9678902495342, 618.9678903816401, 840.6897023292908, 897.5668787664976, 897.566878789376, 897.5668788571334, 1114.2126184036135, 1114.2126184766944, 1328.7437486991935]


Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

extracting frequencies for qpoint [0. 0. 0.]
phonon frequencies are (THz): [0.8821485974191188, 0.8821485982276737, 1.181199168223404, 1.1811991683713499, 1.1811991688139027, 2.1120509272414676, 2.224716245546605, 2.224716245627786, 2.224716245869089, 3.448731336467841, 3.4487313366778842, 3.4557770670929258, 3.7945127522326394, 3.7945127523001805, 3.794512752322955, 7.346760732351766, 7.346760732414707, 7.78807039896586, 7.788070399065135, 8.763951852673788, 9.203817055624388]
audible frequencies are (Hz): [459.97161296278546, 459.9716133660522, 609.1230851514227, 609.1230852252107, 609.123085445934, 1073.3853999616817, 1129.577227466369, 1129.5772275068582, 1129.577227627208, 1740.0547540633356, 1740.0547541680946, 1743.5688122125966, 1912.5132351760287, 1912.513235209715, 1912.5132352210737]


Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

extracting frequencies for qpoint [0. 0. 0.]
phonon frequencies are (THz): [1.3744775985702402, 1.374597747730219, 1.7193690120730578, 2.458053980390216, 2.464248317162893, 3.325658605647117, 3.6311469154152647, 3.9551639930039695, 4.82908457289989, 4.970609723752224, 5.363934110533108, 6.1847044101091235, 6.683178035359772, 7.027900415750076, 7.252355201896318, 7.719661368578635, 10.548805076117752, 11.180071714841194, 11.551171572492455, 12.042461279145224]
audible frequencies are (Hz): [705.5207022869073, 705.5806266804467, 877.5352947714375, 1245.9544227196202, 1249.0438481849928, 1678.6722295664995, 1831.034524063363, 1992.6380415107294]


Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

extracting frequencies for qpoint [0. 0. 0.]
phonon frequencies are (THz): [1.3817563691566404, 1.3817563692725952, 4.317842234854692]
audible frequencies are (Hz): [709.1509891168744, 709.1509891747068, 2173.5238146337774]


Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

extracting frequencies for qpoint [0. 0. 0.]
phonon frequencies are (THz): [2.4609863880016514, 2.6692102896738796, 3.1223842035469382, 3.6305308634182283, 3.6305308634360998, 3.9720845254726647, 5.064129283079707, 5.23274244873493, 5.390123145989248, 5.7287784452324555, 6.223748974555539, 6.495150774138142, 6.503234189891305, 6.503234189910901, 13.082929946634557, 13.082929946707631, 13.51347722766403, 13.971425340806988, 15.178574503824095, 15.260078077380902]
audible frequencies are (Hz): [1247.4169610158235, 1351.2686319748473, 1577.2891215190355, 1830.7272681298414, 1830.7272681387544, 2001.0771570794914]


# Cs3Sb
![](./Cs3Sb.png)
# K2TeCl6
![](./K2TeCl6.png)
# BaAg2GeS4
![](./BaAg2GeS4.png)
# CuCl
![](./CuCl.png)
# CaPbF6
![](./CaPbF6.png)

In [21]:
# specify Materials Project ID or provide your own mesh.yaml or band.yaml
# have in gamma-point mode or band dispersion mode
# gamma point mode: play chord, temperature slider, keyboard of notes
# band dispersion mode: play as moving through reciprocal space, or select position to play chord

# host as a web app
# use as an education outreach tool
# use for undergraduate education
# use for musical performance?
# learning points: role of defects